## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-01-13-05-35 +0000,wapo,The chairs from that viral photo of the Bidens...,https://www.washingtonpost.com/lifestyle/2026/...
1,2026-02-01-13-05-34 +0000,nypost,"Trump threatens to sue Michael Wolff, Epstein ...",https://nypost.com/2026/02/01/us-news/trump-th...
2,2026-02-01-13-02-00 +0000,wsj,French Firm to Sell Division That Helps ICE Tr...,https://www.wsj.com/world/europe/french-firm-t...
3,2026-02-01-13-00-00 +0000,nypost,To the moon! Inside the outer space hotels pre...,https://nypost.com/2026/02/01/us-news/outer-sp...
4,2026-02-01-13-00-00 +0000,wsj,The Hottest Commodity in the Mining Business: ...,https://www.wsj.com/business/the-hottest-commo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2459/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,21
24,ice,15
11,epstein,14
193,iran,9
537,judge,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
75,2026-02-01-04-26-00 +0000,wsj,President Trump joked during a speech that he ...,https://www.wsj.com/politics/policy/trump-joke...,58
25,2026-02-01-11-00-00 +0000,wapo,Trump’s immigration crackdown means more child...,https://www.washingtonpost.com/immigration/202...,55
94,2026-02-01-02-00-00 +0000,wsj,An Abu Dhabi royal backed a secret $500 millio...,https://www.wsj.com/politics/policy/spy-sheikh...,54
137,2026-01-31-20-52-55 +0000,cbc,"Judge orders U.S. to free 5-year-old boy, fath...",https://www.cbc.ca/news/world/father-son-relea...,53
125,2026-01-31-22-14-54 +0000,bbc,Judge orders release of five-year-old detained...,https://www.bbc.com/news/articles/cm2ygq5p2l3o...,52


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
75,58,2026-02-01-04-26-00 +0000,wsj,President Trump joked during a speech that he ...,https://www.wsj.com/politics/policy/trump-joke...
137,53,2026-01-31-20-52-55 +0000,cbc,"Judge orders U.S. to free 5-year-old boy, fath...",https://www.cbc.ca/news/world/father-son-relea...
121,47,2026-01-31-22-44-01 +0000,nypost,UK Prime Minister calls on ex-Prince Andrew to...,https://nypost.com/2026/01/31/world-news/uk-pr...
94,33,2026-02-01-02-00-00 +0000,wsj,An Abu Dhabi royal backed a secret $500 millio...,https://www.wsj.com/politics/policy/spy-sheikh...
49,29,2026-02-01-09-30-50 +0000,nypost,"Texas girl, 10, shot by crazed driver during r...",https://nypost.com/2026/02/01/us-news/girl-sho...
96,29,2026-02-01-01-37-34 +0000,nypost,"Sean Penn, Robert De Niro lead Hollywood A-lis...",https://nypost.com/2026/01/31/us-news/hollywoo...
193,29,2026-01-31-13-30-00 +0000,nypost,Gaza ‘doctor’ who slammed Israel in NY Times o...,https://nypost.com/2026/01/31/world-news/gaza-...
26,28,2026-02-01-10-54-14 +0000,nypost,Iran’s supreme leader warns any US attack woul...,https://nypost.com/2026/02/01/world-news/iran-...
155,26,2026-01-31-18-50-44 +0000,nyt,"New Epstein Files Name Elon Musk, Bill Gates a...",https://www.nytimes.com/2026/01/31/us/epstein-...
145,24,2026-01-31-20-17-52 +0000,latimes,Man sentenced for throwing Molotov cocktail at...,https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
